<a href="https://colab.research.google.com/github/450586509/practical-ml/blob/master/transformers_ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install conllu 
!pip install seqeval

     |████████████████████████████████| 1.3MB 13.5MB/s 
     |████████████████████████████████| 890kB 45.6MB/s 
     |████████████████████████████████| 2.9MB 43.9MB/s 
     |████████████████████████████████| 1.1MB 50.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=f57bdb0ece3eab7cea497a1c4e57026b4476db0fc6ac723bd16897c4d61d37ee
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 51kB 6.2MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=2f4ae8e742f9951f837fb88b3ae498a9508baac8bc334e267f4956d3a3e56f6b
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval


## 准备数据

In [2]:

!wget http://www.xwwlhxf.com/source/task.py
!wget http://www.xwwlhxf.com/source/utils_ner.py
!wget http://www.xwwlhxf.com/source/data-input.zip
!unzip data-input.zip

--2020-11-10 11:17:37--  http://www.xwwlhxf.com/source/task.py
Resolving www.xwwlhxf.com (www.xwwlhxf.com)... 129.204.205.246
Connecting to www.xwwlhxf.com (www.xwwlhxf.com)|129.204.205.246|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5670 (5.5K) [application/octet-stream]
Saving to: ‘task.py’

task.py             100%[===================>]   5.54K  --.-KB/s    in 0s      

2020-11-10 11:17:38 (80.2 MB/s) - ‘task.py’ saved [5670/5670]

--2020-11-10 11:17:38--  http://www.xwwlhxf.com/source/utils_ner.py
Resolving www.xwwlhxf.com (www.xwwlhxf.com)... 129.204.205.246
Connecting to www.xwwlhxf.com (www.xwwlhxf.com)|129.204.205.246|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15322 (15K) [application/octet-stream]
Saving to: ‘utils_ner.py’

utils_ner.py        100%[===================>]  14.96K  69.8KB/s    in 0.2s    

2020-11-10 11:17:39 (69.8 KB/s) - ‘utils_ner.py’ saved [15322/15322]

--2020-11-10 11:17:39--  http://www.xwwlhxf.com

In [3]:
import logging
import os
import sys
from dataclasses import dataclass, field
from importlib import import_module
from typing import Dict, List, Optional, Tuple

import numpy as np
from seqeval.metrics import accuracy_score, f1_score, precision_score, recall_score
from torch import nn

In [4]:
from transformers import (
    AutoConfig,
    AutoModelForTokenClassification,
    AutoTokenizer,
    EvalPrediction,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    set_seed,
)
from utils_ner import Split, TokenClassificationDataset, TokenClassificationTask
logger = logging.getLogger(__name__)

In [5]:
@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    task_type: Optional[str] = field(
        default="NER", metadata={"help": "Task type to fine tune in training (e.g. NER, POS, etc)"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    use_fast: bool = field(default=False, metadata={"help": "Set this flag to use fast tokenization."})
    # If you want to tweak more attributes on your tokenizer, you should do it in a distinct script,
    # or just modify its tokenizer_config.json.
    cache_dir: Optional[str] = field(
        default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
    )


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    data_dir: str = field(
        metadata={"help": "The input data dir. Should contain the .txt files for a CoNLL-2003-formatted task."}
    )
    labels: Optional[str] = field(
        default=None,
        metadata={"help": "Path to a file containing all labels. If not specified, CoNLL-2003 labels are used."},
    )
    max_seq_length: int = field(
        default=128,
        metadata={
            "help": "The maximum total input sequence length after tokenization. Sequences longer "
                    "than this will be truncated, sequences shorter will be padded."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )



In [6]:
sys.argv = ["t.py", "--data_dir", "./data-input", "--model_name_or_path", "bert-base-uncased", "--output_dir",
            "./output", "--labels", "./data-input/labels.txt", "--do_train", "--do_predict", "--max_seq_length", "256",
            "--overwrite_output_dir", "--overwrite_cache"]
parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))
if len(sys.argv) == 2 and sys.argv[1].endswith(".json"):
    # If we pass only one argument to the script and it's the path to a json file,
    # let's parse it to get our arguments.
    model_args, data_args, training_args = parser.parse_json_file(json_file=os.path.abspath(sys.argv[1]))
else:
    model_args, data_args, training_args = parser.parse_args_into_dataclasses()
    print(f"model_args={model_args}")
    print(f"data_args={data_args}")
    print(f"training_args={training_args}")

if (
        os.path.exists(training_args.output_dir)
        and os.listdir(training_args.output_dir)
        and training_args.do_train
        and not training_args.overwrite_output_dir
):
    raise ValueError(
        f"Output directory ({training_args.output_dir}) already exists and is not empty. Use --overwrite_output_dir to overcome."
    )




model_args=ModelArguments(model_name_or_path='bert-base-uncased', config_name=None, task_type='NER', tokenizer_name=None, use_fast=False, cache_dir=None)
data_args=DataTrainingArguments(data_dir='./data-input', labels='./data-input/labels.txt', max_seq_length=256, overwrite_cache=True)
training_args=TrainingArguments(output_dir='./output', overwrite_output_dir=True, do_train=True, do_eval=False, do_predict=True, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Nov10_11-28-42_dce86478d594', logging_first_step=False, logging_steps=500, save_steps=500, save_total_limit=

In [9]:
module = import_module("task")
print(f"task_type={model_args.task_type}")
try:
    token_classification_task_clazz = getattr(module, model_args.task_type)
    token_classification_task: TokenClassificationTask = token_classification_task_clazz()
except AttributeError:
    raise ValueError(
        f"Task {model_args.task_type} needs to be defined as a TokenClassificationTask subclass in {module}. "
        f"Available tasks classes are: {TokenClassificationTask.__subclasses__()}"
    )
print(f"token_classification_task_clazz={token_classification_task_clazz}")
print(f"token_classification_task={token_classification_task}")

task_type=NER
token_classification_task_clazz=<class 'task.NER'>
token_classification_task=<task.NER object at 0x7f31f9525668>


In [11]:
set_seed(training_args.seed)
print(f"data_args.labels={data_args.labels}")
labels = token_classification_task.get_labels(data_args.labels)
label_map: Dict[int, str] = {i: label for i, label in enumerate(labels)}
num_labels = len(labels)
print(f"labels={labels}")

data_args.labels=./data-input/labels.txt
labels=['O', 'B-Chemical', 'I-Chemical', 'B-Disease', 'I-Disease']


In [12]:
print(f"config_name={model_args.config_name}")
print(f"config_name={model_args.model_name_or_path}")

config_name=None
config_name=bert-base-uncased


In [14]:
config = AutoConfig.from_pretrained(
    model_args.config_name if model_args.config_name else model_args.model_name_or_path,
    num_labels=num_labels,
    id2label=label_map,
    label2id={label: i for i, label in enumerate(labels)},
    cache_dir=model_args.cache_dir,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
    cache_dir=model_args.cache_dir,
    use_fast=model_args.use_fast,
)


In [15]:
# Get datasets
print(f"train data_dir={data_args.data_dir}")
train_dataset = (
    TokenClassificationDataset(
        token_classification_task=token_classification_task,
        data_dir=data_args.data_dir,
        tokenizer=tokenizer,
        labels=labels,
        model_type=config.model_type,
        max_seq_length=data_args.max_seq_length,
        overwrite_cache=data_args.overwrite_cache,
        mode=Split.train,
    )
    if training_args.do_train
    else None
)
eval_dataset = (
    TokenClassificationDataset(
        token_classification_task=token_classification_task,
        data_dir=data_args.data_dir,
        tokenizer=tokenizer,
        labels=labels,
        model_type=config.model_type,
        max_seq_length=data_args.max_seq_length,
        overwrite_cache=data_args.overwrite_cache,
        mode=Split.dev,
    )
    if training_args.do_eval
    else None
)


train data_dir=./data-input


In [16]:
model = AutoModelForTokenClassification.from_pretrained(
    model_args.model_name_or_path,
    from_tf=bool(".ckpt" in model_args.model_name_or_path),
    config=config,
    cache_dir=model_args.cache_dir,
)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [17]:
def align_predictions(predictions: np.ndarray, label_ids: np.ndarray) -> Tuple[List[int], List[int]]:
    preds = np.argmax(predictions, axis=2)

    batch_size, seq_len = preds.shape

    out_label_list = [[] for _ in range(batch_size)]
    preds_list = [[] for _ in range(batch_size)]

    for i in range(batch_size):
        for j in range(seq_len):
            if label_ids[i, j] != nn.CrossEntropyLoss().ignore_index:
                out_label_list[i].append(label_map[label_ids[i][j]])
                preds_list[i].append(label_map[preds[i][j]])

    return preds_list, out_label_list


def compute_metrics(p: EvalPrediction) -> Dict:
    preds_list, out_label_list = align_predictions(p.predictions, p.label_ids)
    return {
        "accuracy_score": accuracy_score(out_label_list, preds_list),
        "precision": precision_score(out_label_list, preds_list),
        "recall": recall_score(out_label_list, preds_list),
        "f1": f1_score(out_label_list, preds_list),
    }


In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

# Training
if training_args.do_train:
    trainer.train(
        model_path=model_args.model_name_or_path if os.path.isdir(model_args.model_name_or_path) else None
    )
    trainer.save_model()
    # For convenience, we also re-save the tokenizer to the same directory,
    # so that you can share your model easily on huggingface.co/models =)
    if trainer.is_world_master():
        tokenizer.save_pretrained(training_args.output_dir)


Step,Training Loss
500,0.135315


KeyboardInterrupt: ignored

11/09/2020 11:02:51 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
11/09/2020 11:02:51 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='./output', overwrite_output_dir=True, do_train=True, do_eval=False, do_predict=True, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Nov09_11-02-51_3d424a53a4f6', logging_first_step=False, logging_steps=500, save_steps=500, save_total_limit=None, no_cuda=False, seed=42, fp16=False, fp16_opt_level='O1', local_rank=-1, tpu_num_cor

model_args=ModelArguments(model_name_or_path='bert-base-uncased', config_name=None, task_type='NER', tokenizer_name=None, use_fast=False, cache_dir=None)
data_args=DataTrainingArguments(data_dir='./data-input', labels='./data-input/labels.txt', max_seq_length=256, overwrite_cache=True)
training_args=TrainingArguments(output_dir='./output', overwrite_output_dir=True, do_train=True, do_eval=False, do_predict=True, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Nov09_11-02-51_3d424a53a4f6', logging_first_step=False, logging_steps=500, save_steps=500, save_total_limit=

11/09/2020 11:02:52 - INFO - filelock -   Lock 139849966730040 released on /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517.lock
11/09/2020 11:02:52 - INFO - filelock -   Lock 139849966707096 acquired on /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock


11/09/2020 11:02:52 - INFO - filelock -   Lock 139849966707096 released on /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock
11/09/2020 11:02:52 - INFO - filelock -   Lock 139849966339688 acquired on ./data-input/cached_train_BertTokenizer_256.lock
11/09/2020 11:02:52 - INFO - utils_ner -   Creating features from dataset file at ./data-input



train data_dir=./data-input


11/09/2020 11:02:52 - INFO - utils_ner -   Writing example 0 of 9141
11/09/2020 11:02:52 - INFO - utils_ner -   *** Example ***
11/09/2020 11:02:52 - INFO - utils_ner -   guid: train-1
11/09/2020 11:02:52 - INFO - utils_ner -   tokens: [CLS] se ##leg ##ili ##ne - induced post ##ural h ##yp ##ote ##ns ##ion in parkinson ' s disease : a longitudinal study on the effects of drug withdrawal . [SEP]
11/09/2020 11:02:52 - INFO - utils_ner -   input_ids: 101 7367 23115 18622 2638 1011 10572 2695 11137 1044 22571 12184 3619 3258 1999 20310 1005 1055 4295 1024 1037 20134 2817 2006 1996 3896 1997 4319 10534 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

### 验证

In [ ]:
results = {}
if training_args.do_eval:
    logger.info("*** Evaluate ***")

    result = trainer.evaluate()

    output_eval_file = os.path.join(training_args.output_dir, "eval_results.txt")
    if trainer.is_world_master():
        with open(output_eval_file, "w") as writer:
            logger.info("***** Eval results *****")
            for key, value in result.items():
                logger.info("  %s = %s", key, value)
                writer.write("%s = %s\n" % (key, value))

        results.update(result)


### predict

In [ ]:
# Predict
if training_args.do_predict:
    test_dataset = TokenClassificationDataset(
        token_classification_task=token_classification_task,
        data_dir=data_args.data_dir,
        tokenizer=tokenizer,
        labels=labels,
        model_type=config.model_type,
        max_seq_length=data_args.max_seq_length,
        overwrite_cache=data_args.overwrite_cache,
        mode=Split.test,
    )

    predictions, label_ids, metrics = trainer.predict(test_dataset)
    preds_list, _ = align_predictions(predictions, label_ids)

    output_test_results_file = os.path.join(training_args.output_dir, "test_results.txt")
    if trainer.is_world_master():
        with open(output_test_results_file, "w") as writer:
            for key, value in metrics.items():
                logger.info("  %s = %s", key, value)
                writer.write("%s = %s\n" % (key, value))
    output_test_predictions_file = os.path.join(training_args.output_dir, "test_predictions.txt")
    if trainer.is_world_master():
        with open(output_test_predictions_file, "w") as writer:
            with open(os.path.join(data_args.data_dir, "test.txt"), "r") as f:
                token_classification_task.write_predictions_to_file(writer, f, preds_list)
    print(f"result={result}")
